# Assignment 10

Consider the reservoir shown below with the given properties that has been discretized into 4 equal grid blocks.

![image](images/grid.png)

Below is a skeleton of a Python class that can be used to solve for the pressures in the reservoir.  The class is actually written generally enough that it can account for an arbitrary number of grid blocks, but we will only test cases with 4.  The class takes a Python dictonary of input parameters as an initialization argument.  An example of a complete set of input parameters is shown in the `setup()` function of the tests below.

Several simple useful functions are already implemented, your task is to implement the functions `fill_matrices()` and `solve_one_step()`.  `fill_matrices()` should correctly populate the $A$, $I$ matrices as well as the vector $\vec{p}_B$.  These should also correctly account for the application of boundary conditions.  Only the boundary conditions shown in the figure will be tested, but in preparation for future assignments, you may wish to add the logic to the code such that arbitrary pressure/no flow boundary conditions can be applied to either side of the one-dimensional reservoir. `solve_one_step()` should solve a single time step for either the explicit or implicit methods depending on which is specified in the input parameters. The $\vec{p}{}^{n+1}$ values should be stored in the class attribute `self.p`.  If this is implemented correctly, you will be able to then use the `solve()` function to solve the problem up to the `'number of time steps'` value in the input parameters.

Once you have the tests passing, you might like to experiment with viewing several plots with different time steps, explicit vs. implicit, number of grid blocks, etc.  To assist in giving you a feel for how they change the character of the approximate solution.  I have implemented a simple plot function that might help for this.

In [1]:
import numpy as np
import scipy.sparse
import scipy.sparse.linalg

In [2]:
class OneDimReservoir():
    
    def __init__(self, inputs):
        '''
            Class for solving one-dimensional reservoir problems with
            finite differences.
        '''
        
        #stores input dictionary as class attribute
        self.inputs = inputs
        
        #computes delta_x
        self.delta_x = self.inputs['reservoir']['length'] / float(self.inputs['numerical']['number of grids'])
        
        #gets delta_t from inputs
        self.delta_t = self.inputs['numerical']['time step']
        
        #computes \eta
        self.compute_eta()
        
        #calls fill matrix method (must be completely implemented to work)
        self.fill_matrices()
        
        #applies the initial reservoir pressues to self.p
        self.apply_initial_conditions()
        
        #create an empty list for storing data if plots are requested
        if 'plots' in self.inputs:
            self.p_plot = []
            
        return
        
    
    def compute_alpha(self):
        '''
            Computes the constant \alpha.
        '''
        
        c_t = self.inputs['fluid']['compressibility']
        mu = self.inputs['fluid']['viscosity']
        phi = self.inputs['reservoir']['porosity']
        k = self.inputs['reservoir']['permeability']
        
        return k / mu / phi / c_t
    
    def compute_eta(self):
        '''
            Computes the constant \eta
        '''
        
        alpha = self.compute_alpha()
        factor = self.inputs['conversion factor']
        dx = self.delta_x
        dt = self.delta_t
        
        self.eta = alpha * dt / dx ** 2 * factor
    
    def fill_matrices(self):
        '''
            Fills the matrices A, I, and \vec{p}_B and applies boundary
            conditions.
        '''
        
        N = self.inputs['numerical']['number of grids']
        
        self.p = np.zeros(N)
        
        diagonal = np.ones(N) * 2
        self.pB = np.zeros(N)
        
        self.apply_boundary_conditions(diagonal, self.pB)
        
        diagonals = [diagonal, -np.ones(N - 1), -np.ones(N - 1)]
        
        self.A = scipy.sparse.diags(diagonals, [0, -1, 1])
        self.I = scipy.sparse.eye(N)
        
        return
        
    def apply_boundary_conditions(self, D, Q):
        '''
            Applies boundary conditions (called by fill_matrices).
        '''
        
        N = self.inputs['numerical']['number of grids']
        
        bc = self.inputs['boundary conditions']
            
        if bc['left']['type'] == 'prescribed pressure':

            D[0] = 3
            Q[0] = 2 * bc['left']['value']  * self.eta 

        elif bc['left']['type'] == 'prescribed flux':

            mu = self.inputs['fluid']['viscosity']
            k = self.inputs['reservoir']['permeability']
            dx = self.delta_x

            D[0] = 1
            Q[0] = bc['left']['value'] * mu / dx / k * self.eta

        if bc['right']['type'] == 'prescribed pressure':


            D[N-1] = 3
            Q[N-1] = 2 * bc['left']['value'] * self.eta

        elif bc['right']['type'] == 'prescribed flux':

            mu = self.inputs['fluid']['viscosity']
            k = self.inputs['reservoir']['permeability']
            dx = self.delta_x

            D[N-1] = 1
            Q[N-1] = bc['left']['value'] * mu / dx / k * self.eta
            
        return
            
                
    def apply_initial_conditions(self):
        '''
            Applies initial pressures to self.p
        '''
        
        N = self.inputs['numerical']['number of grids']
        
        self.p = np.ones(N) * self.inputs['initial conditions']['pressure']
        
        return
                
                
    def solve_one_step(self):
        '''
            Solve one time step using either the implicit or explicit method
        '''
        
        I = self.I
        A = self.A
        pB = self.pB
        eta = self.eta
        
        if self.inputs['numerical']['solver'] == 'explicit':
            self.p = (I - eta * A).dot(self.p) + pB
        elif self.inputs['numerical']['solver'] == 'implicit':
            self.p = scipy.sparse.linalg.spsolve(I + eta * A, self.p + pB)
            
        return
            
            
    def solve(self):
        '''
            Solves until "number of time steps"
        '''
        
        for i in range(self.inputs['numerical']['number of time steps']):
            self.solve_one_step()
            
            if i % self.inputs['plots']['frequency'] == 0:
                self.p_plot += [self.get_solution()]
                
        return
                
    def plot(self):
        '''
           Crude plotting function.  Plots pressure as a function of grid block #
        '''
        
        if self.p_plot is not None:
            for i in range(len(self.p_plot)):
                plt.plot(self.p_plot[i])
        
        return
            
    def get_solution(self):
        '''
            Returns solution vector
        '''
        return self.p

# DO NOT EDIT BELOW THIS LINE

In [3]:
def setup():
    
    inputs = {
        'conversion factor': 6.33e-3,
        'fluid': {
            'compressibility': 1e-6, #psi^{-1}
            'viscosity': 1 #cp
        },
        'reservoir': {
            'permeability': 50, #mD
            'porosity': 0.2,
            'length': 10000, #ft
        },
        'initial conditions': {
            'pressure': 1000 #psi
        },
        'boundary conditions': {
            'left': {
                'type': 'prescribed pressure',
                'value': 2000 #psi
            },
            'right': {
                'type': 'prescribed flux',
                'value': 0 #ft^3/day
            }
        },
        'numerical': {
            'solver': 'implicit',
            'number of grids': 4,
            'time step': 1, #day
            'number of time steps' : 3 
        },
        'plots': {
            'frequency': 1
        }
    }
    
    return inputs
  
def test_eta():
    
    parameters = setup()
    
    problem = OneDimReservoir(parameters)
    
    np.testing.assert_allclose(problem.eta, 0.2532, atol=1e-3)
    
    return

def test_implicit_solve_one_step():
    
    parameters = setup()
    
    implicit = OneDimReservoir(parameters)
    implicit.solve_one_step()
    np.testing.assert_allclose(implicit.get_solution(), 
                               np.array([1295.1463, 1051.1036, 1008.8921, 1001.7998]), 
                               atol=0.5)
    return

def test_explicit_solve_one_step():
    
    parameters = setup()
    
    parameters['numerical']['solver'] = 'explicit'
    
    explicit = OneDimReservoir(parameters)
    
    explicit.solve_one_step()

    np.testing.assert_allclose(explicit.get_solution(), 
                           np.array([ 1506., 1000.,  1000.,  1000.004]), 
                           atol=0.5)
    return 

def test_implicit_solve():
    
    parameters = setup()
    
    implicit = OneDimReservoir(parameters)
    implicit.solve()
    np.testing.assert_allclose(implicit.get_solution(), 
                               np.array([1582.9, 1184.8, 1051.5, 1015.9]), 
                               atol=0.5)
    return

def test_explicit_solve():
    
    parameters = setup()
    
    parameters['numerical']['solver'] = 'explicit'
    
    explicit = OneDimReservoir(parameters)
    
    explicit.solve()

    np.testing.assert_allclose(explicit.get_solution(), 
                           np.array([1689.8, 1222.3, 1032.4, 1000.0]), 
                           atol=0.5)
    return 